In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import re

In [10]:
# Load the dataset
url = "training.1600000.processed.noemoticon.csv"  # Change this to the actual file path
df = pd.read_csv(url, encoding='latin1', header=None, 
                 names=["sentiment", "id", "date", "query", "user", "text"])

# Let's inspect the first few rows
df.head()

# Clean the text data (remove URLs, user handles, and other noise)
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\S+', '', text)  # Remove mentions
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetical characters
    return text.lower()  # Convert to lowercase

df['text'] = df['text'].apply(clean_text)

# Encode labels: 0 for negative, 1 for positive sentiment
X = df['text'].values
y = df['sentiment'].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)  # Consider top 5000 words
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# Pad sequences to ensure uniform length
X_train_pad = pad_sequences(X_train_seq, maxlen=100)
X_val_pad = pad_sequences(X_val_seq, maxlen=100)

In [11]:
# Build the CNN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))  # Embedding layer
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))  # Convolutional layer
model.add(MaxPooling1D(pool_size=4))  # Max-pooling layer
model.add(Flatten())  # Flatten the output
model.add(Dense(1, activation='sigmoid'))  # Output layer (binary classification)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          640000    
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           82048     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 128)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 1)                 3073      
                                                                 
Total params: 725,121
Trainable params: 725,121
Non-trainable params: 0
__________________________________________________

In [12]:
# Train the model
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

Epoch 1/5
40000/40000 [==============================] - 1396s 35ms/step - loss: -43078844416.0000 - accuracy: 3.1250e-06 - val_loss: -158648270848.0000 - val_accuracy: 0.0000e+00
Epoch 2/5
40000/40000 [==============================] - 1543s 39ms/step - loss: -519622000640.0000 - accuracy: 0.0000e+00 - val_loss: -1065137799168.0000 - val_accuracy: 0.0000e+00
Epoch 3/5
40000/40000 [==============================] - 1398s 35ms/step - loss: -2049458700288.0000 - accuracy: 0.0000e+00 - val_loss: -3343516958720.0000 - val_accuracy: 0.0000e+00
Epoch 4/5
  177/40000 [..............................] - ETA: 21:28 - loss: -3292910583808.0000 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val_pad, y_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")